#### Imports

In [2]:
import numpy as np
import cv2 as cv
import os

#### Main utility functions

In [8]:
def get_puzzle(img, corners=[]):
    if len(corners) == 0:
        low = np.array([154, 54, 0])
        high= np.array([255, 255, 255])

        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask = cv.inRange(img_hsv, low, high)
        
        kernel = np.ones((3, 3), np.uint8)
        mask = cv.erode(mask, kernel)

        mean = np.mean(mask)
        l = 0.66 * mean
        u = 1.33 * mean
        edges = cv.Canny(mask, l, u)

        contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        top_left = []
        bottom_left = []
        top_right = []
        bottom_right = []

        for i in range(len(contours)):
            if(len(contours[i]) >3):
                possible_top_left = None
                possible_bottom_right = None
                for point in contours[i].squeeze():
                    if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                        possible_top_left = point

                    if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                        possible_bottom_right = point

                diff = np.diff(contours[i].squeeze(), axis = 1)
                possible_top_right = contours[i].squeeze()[np.argmin(diff)]
                possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
                current_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))

                # i want red squares
                # TODO: Use the thid maximum instead of experimental values
                if  current_area >= 15000.0 and current_area <= 20000.0:
                    top_left.append(possible_top_left)
                    bottom_right.append(possible_bottom_right)
                    top_right.append(possible_top_right)
                    bottom_left.append(possible_bottom_left)

        top_left = np.array(top_left)
        top_right = np.array(top_right)
        bottom_left = np.array(bottom_left)
        bottom_right = np.array(bottom_right)

        dist = []
        for tl in top_left:
            dist.append(np.sqrt(tl[0]**2 + tl[1]**2))
        dist = np.array(dist)

        indices = np.argsort(dist)

        top_left = top_left[indices]
        top_right = top_right[indices]
        bottom_left = bottom_left[indices]
        bottom_right = bottom_right[indices]

        c1 = top_left[0]
        c2 = top_right[3]
        c3 = bottom_right[7]
        c4 = bottom_left[4]

        corners = [c1, c2, c3, c4]
  

    height, width = 1500, 1500

    puzzle = np.array(corners, dtype = "float32")
    destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    result = cv.warpPerspective(img, M, (width, height))

    return (result, corners)

horizontal_lines = []
for i in range(0, 1501, 100):
    l = []
    l.append((0, i))
    l.append((1500, i))
    horizontal_lines.append(l)

vertical_lines = []
for i in range(0, 1501, 100):
    l = []
    l.append((i, 0))
    l.append((i, 1500))
    vertical_lines.append(l)

def get_puzzle_configuration(puzzle, last_positions, horizontal_lines, vertical_lines):
    positions = ''
    
    for i in range(len(horizontal_lines)-1):
        for j in range(len(vertical_lines)-1):
            y_min = vertical_lines[j][0][0] + 10
            y_max = vertical_lines[j + 1][1][0] - 10
            x_min = horizontal_lines[i][0][1] + 10
            x_max = horizontal_lines[i + 1][1][1] - 10

            patch = puzzle[x_min:x_max, y_min:y_max].copy()

            low = np.array([80, 0, 117])
            high= np.array([105, 148, 255])

            patch_mask = cv.inRange(cv.cvtColor(patch, cv.COLOR_BGR2HSV), low, high)
            kernel = np.ones((3, 3), np.uint8)
            patch_mask = cv.erode(patch_mask, kernel) 

            mean = np.mean(patch_mask)

            if int(mean) > 10:
                position = f'{i+1}{chr(j+65)} A'
                if position not in last_positions:
                    positions += position + '\n'
                    last_positions.append(position)

    positions += '10'
    return (positions, last_positions)

def letter_classifier(patch):
        maximum = -np.inf
        letter = 'Q'

        for i in range(65, 91):
            # K Q Y => jump over
            if i in [75, 81, 89]:
                 continue

            template = cv.imread('./templates/v7/' + str(chr(i)) + '.jpg')
            template = cv.cvtColor(template,cv.COLOR_BGR2GRAY)

            corr = cv.matchTemplate(patch, template, cv.TM_CCOEFF_NORMED)
            corr = np.max(corr)
            
            if corr > maximum:
                maximum = corr
                letter = chr(i)

        return letter

def get_puzzle_configuration_wletter(puzzle, last_positions, horizontal_lines, vertical_lines):
    positions = ''

    for i in range(len(horizontal_lines)-1):
        for j in range(len(vertical_lines)-1):
            y_min = vertical_lines[j][0][0] + 10
            y_max = vertical_lines[j + 1][1][0] - 10
            x_min = horizontal_lines[i][0][1] + 10
            x_max = horizontal_lines[i + 1][1][1] - 10

            patch = puzzle[x_min:x_max, y_min:y_max].copy()
            patch_gray = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)

            low = np.array([80, 0, 117])
            high= np.array([105, 148, 255])

            patch_mask = cv.inRange(cv.cvtColor(patch, cv.COLOR_BGR2HSV), low, high)
            kernel = np.ones((3, 3), np.uint8)
            patch_mask = cv.erode(patch_mask, kernel) 

            mean = np.mean(patch_mask)

            if int(mean) > 10:

                position = f'{i+1}{chr(j+65)}'

                if position not in last_positions:
                    letter = letter_classifier(patch_gray)

                    if letter == 'W': letter = '?'

                    positions += position + ' ' + letter + '\n'
                    last_positions.append(position)
                    
    positions += '10'
            
    return (positions, last_positions)

def compute_score(game_matrix, round_matrix):
    bonus = np.array([
       [9, 1, 1, 2, 1, 1, 1, 9, 1, 1, 1, 2, 1, 1, 9],
       [1, 8, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 8, 1],
       [1, 1, 8, 1, 1, 1, 2, 1, 2, 1, 1, 1, 8, 1, 1],
       [2, 1, 1, 8, 1, 1, 1, 2, 1, 1, 1, 8, 1, 1, 2],
       [1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1],
       [1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1],
       [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],
       [9, 1, 1, 2, 1, 1, 1, 8, 1, 1, 1, 2, 1, 1, 9],
       [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],
       [1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1],
       [2, 1, 1, 8, 1, 1, 1, 2, 1, 1, 1, 8, 1, 1, 2],
       [1, 1, 8, 1, 1, 1, 2, 1, 2, 1, 1, 1, 8, 1, 1],
       [1, 8, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 8, 1],
       [9, 1, 1, 2, 1, 1, 1, 9, 1, 1, 1, 2, 1, 1, 9]
    ])


    letter_score = {
        'A': 1,
        'B': 9,
        'C': 1,
        'D': 2,
        'E': 1,
        'F': 8,
        'G': 9,
        'H': 10,
        'I': 1,
        'J': 10,
        'L': 1,
        'M': 4,
        'N': 1,
        'O': 1,
        'P': 2,
        'R': 1,
        'S': 1,
        'T': 1,
        'U': 1,
        'V': 8,
        'X': 10,
        'Z': 10,
        '?': 0,
        'W': 0 
    }

    def compute_word_score(l_pos, new_letters_pos):
        word_score = 0
        m_word = 1
        for l in l_pos:
            l_is_new = np.any(np.all(np.array(l) == new_letters_pos, axis=1))
            if bonus[l] < 8:
                l_score = letter_score[game_matrix[l]]

                if l_is_new:
                    word_score += l_score * bonus[l]
                else:
                    word_score += l_score
            else:
                word_score += letter_score[game_matrix[l]]

                if l_is_new:
                    if bonus[l] == 8:
                        m_word *= 2
                    if bonus[l] == 9:
                        m_word *= 3

        return word_score * m_word
    
    def check_if_new_word(letters_pos, new_letters_pos):
        n = len(letters_pos)

        arr = np.zeros(n, dtype='bool')
        for i in range(n):
            for j in range(new_letters_pos.shape[0]):
                if letters_pos[i][0] == new_letters_pos[j, 0] and letters_pos[i][1] == new_letters_pos[j, 1]:
                    arr[i] = True

        return np.sum(arr)
        
    score = 0
    
    new_letters_pos = np.argwhere(round_matrix != '')

    for (i, j) in new_letters_pos:
        game_matrix[i, j] = round_matrix[i, j]

    lines, cols = np.unique(new_letters_pos[:,0]), np.unique(new_letters_pos[:,1])
    
    # Scan lines and cols
    for i in lines:
        letters_pos = []

        for j in range(0, 15):
            if game_matrix[i, j] != '':
                letters_pos.append((i, j))
            else:
                if len(letters_pos) >= 2:
                    # if letter_positions contain letters from current round
                    new_word = check_if_new_word(letters_pos, new_letters_pos)
                    if new_word:
                        # add up this word with bonus
                        score += compute_word_score(letters_pos, new_letters_pos) 
                letters_pos = [] 

        # if the word ends at the last column        
        if len(letters_pos) >= 2:
            # if letter_positions contain letters from current round
            new_word = check_if_new_word(letters_pos, new_letters_pos)
            if new_word:
                # add up this word with bonus
                score += compute_word_score(letters_pos, new_letters_pos)  
                    
    for j in cols:
        letters_pos = []    

        for i in range(0, 15):
            if game_matrix[i, j] != '':
                letters_pos.append((i, j))
            else:
                if len(letters_pos) >= 2:   
                    # if letter_positions contain letters from current round
                    new_word = check_if_new_word(letters_pos, new_letters_pos)
                    if new_word:
                        # add up this word with bonus
                        score += compute_word_score(letters_pos, new_letters_pos)
                letters_pos = []
        
        # if the word ends at the last row
        if len(letters_pos) >= 2:   
            # if letter_positions contain letters from current round
            new_word = check_if_new_word(letters_pos, new_letters_pos)
            if new_word:
                # add up this word with bonus
                score += compute_word_score(letters_pos, new_letters_pos)


    return (str(score), game_matrix)

def get_puzzle_configuration_wletter_wscore(puzzle, last_positions, horizontal_lines, vertical_lines):
    positions = ''
    round_matrix = np.zeros((15, 15), dtype='str')
    for i in range(len(horizontal_lines)-1):
        for j in range(len(vertical_lines)-1):
            y_min = vertical_lines[j][0][0] + 10
            y_max = vertical_lines[j + 1][1][0] - 10
            x_min = horizontal_lines[i][0][1] + 10
            x_max = horizontal_lines[i + 1][1][1] - 10

            patch = puzzle[x_min:x_max, y_min:y_max].copy()
            patch_gray = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)

            low = np.array([80, 0, 117])
            high= np.array([105, 148, 255])

            patch_mask = cv.inRange(cv.cvtColor(patch, cv.COLOR_BGR2HSV), low, high)
            kernel = np.ones((3, 3), np.uint8)
            patch_mask = cv.erode(patch_mask, kernel) 

            mean = np.mean(patch_mask)

            if int(mean) > 10:

                position = f'{i+1}{chr(j+65)}'

                if position not in last_positions:
                    letter = letter_classifier(patch_gray)

                    if letter == 'W': letter = '?'

                    positions += position + ' ' + letter + '\n'
                    last_positions.append(position)
                    round_matrix[i, j] = letter
            
    return (positions, last_positions, round_matrix)


#### Utility functions - Used only for development. Not needed for testing

In [4]:
def show_image(title, image):
    fx = 0.5
    fy = 0.5

    if image.shape[0] > 4000 and image.shape[1] > 3000:
        fx = 0.2
        fy = 0.2

    image = cv.resize(image, (0,0), fx=fx,fy=fy)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def show_config(result, matrix, lines_horizontal, lines_vertical):
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            if matrix[i, j] != 0: # Task 1; Change matrix[i,j] != '' for task 3
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)
    return result

def show_grid(img, horizontal_lines, vertical_lines):
    img = cv.imread("./antrenare/1_01.jpg")
    puzzle, corners = get_puzzle(img)

    for line in horizontal_lines:
        cv.line(puzzle, line[0], line[1], (0, 255, 0), 5)
    for line in vertical_lines:
        cv.line(puzzle, line[0], line[1], (0, 0, 255), 5)
    show_image('lines', puzzle)

#### Task 1

In [ ]:
path = './antrenare/'
files=sorted(os.listdir(path))
k = 0
for file in files:
    if file[-3:]=='jpg':
        i, j = file.split('.')[0].split('_')
        image = cv.imread(path + file)

        print(f'Start game {i} round {j}')

        if j == '01':
            corners = []
            last_postions = []
        
        puzzle, corners = get_puzzle(image, corners)
        positions, last_postions = get_puzzle_configuration(puzzle, last_postions, horizontal_lines, vertical_lines)

        f = open(f'./evaluare/fisiere_solutie/311_Costea_Razvan_George/{i}_{j}.txt', 'w')
        f.write(positions)
        f.close()

        print(f'End game {i} round {j}')
        

#### Task 2

In [ ]:
path = './antrenare/'
files=sorted(os.listdir(path))

for file in files:
    if file[-3:]=='jpg':
        i, j = file.split('.')[0].split('_')
        image = cv.imread(path + file)

        print(f'Start game {i} round {j}')

        if j == '01':
            corners = []
            last_postions = []
            
        puzzle, corners = get_puzzle(image, corners)
        positions, last_postions = get_puzzle_configuration_wletter(puzzle, last_postions, horizontal_lines, vertical_lines)
        
        f = open(f'./evaluare/fisiere_solutie/311_Costea_Razvan_George/{i}_{j}.txt', 'w')
        f.write(positions)
        f.close()

        print(f'End game {i} round {j}')

#### Task 3

In [ ]:
path = './antrenare/'
files=sorted(os.listdir(path))

for file in files:
    if file[-3:]=='jpg':
        i, j = file.split('.')[0].split('_')
        image = cv.imread(path + file)

        print(f'Start game {i} round {j}')
        matrix = np.zeros((15, 15), dtype='str')

        #First round => New game
        if j == '01':
            corners = []
            last_postions = []
            game_matrix = np.zeros((15, 15), dtype='str')
            
        puzzle, corners = get_puzzle(image, corners)
        positions, last_postions, round_matrix = get_puzzle_configuration_wletter_wscore(puzzle, last_postions, horizontal_lines, vertical_lines)
        
        #Score
        score_result = compute_score(game_matrix, round_matrix)
        positions += score_result[0]
        game_matrix = score_result[1]

        f = open(f'./evaluare/fisiere_solutie/311_Costea_Razvan_George/{i}_{j}.txt', 'w')
        f.write(positions)
        f.close()

        print(f'End game {i} round {j}')